In [70]:
import sys
import os
# Get the current script's directory
current_dir = os.getcwd()

# Get the parent directory
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)
parent_dir

'd:\\algothon2025\\algothon2025'

In [71]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from ta.momentum import RSIIndicator, StochasticOscillator, ROCIndicator, WilliamsRIndicator
from ta.trend import MACD


In [72]:
import numpy as np
import tensorflow as tf
import keras
from keras import layers

In [73]:
from preprocess import COMMISSION_RATE, extract_features2
BEFORE = 20
AHEAD = 40
TA_WINDOW = 50

# Per stock extraction
def split_seq(price_data: pd.DataFrame, stock: int):
    length = price_data[stock].shape[0]
    price_data.index = list(range(length))
    X = []
    y = []

    rsi_series, rsi_series2, macd_signal, macd_signal2, macd_signal3, \
        stoch_sign, stoch_sign2, roc_vals, roc_vals2, roc_vals3, \
        williamR_vals, williamR_vals2 = extract_features2(price_data[stock])
    # print(rsi_series)
    # print(rsi_series.shape)
    for i in range(TA_WINDOW + BEFORE - 1, length - AHEAD):
        # Try 1 price/pct: Accuracy 0.57%, seems to do 0R
        # Try all price/pct: Overfit
        # X.append(price_data.iloc[(i - BEFORE + 1):(i+1), :].pct_change().dropna().to_numpy().reshape((BEFORE - 1, -1)))
        
        # Try feature extractions for single price
        current = []
        for j in range(BEFORE):
            current.append([rsi_series[i-j], rsi_series2[i-j],
                            macd_signal[i-j], macd_signal2[i-j], macd_signal3[i-j],
                            stoch_sign[i-j], stoch_sign2[i-j],
                            roc_vals[i-j], roc_vals2[i-j], roc_vals3[i-j],
                            williamR_vals[i-j], williamR_vals2[i-j]])
        X.append(np.array(current))
        return_pct = (price_data[stock][i+AHEAD] - price_data[stock][i]) / price_data[stock][i]
        if abs(return_pct) < COMMISSION_RATE:
            y.append(0)
        else:
            if return_pct > 0:
                y.append(1)
            else:
                y.append(-1)
    # print(X)
    X_df = np.array(X)
    y_df = np.array(y)
    y_df = tf.one_hot(y_df + 1, depth=3)
    return X_df, y_df


In [74]:
df = pd.read_csv("../prices.txt", sep="\\s+", header=None, index_col=None)
df.index = np.arange(df.shape[0])
df.rename(columns=lambda c: int(c), inplace=True)

In [75]:
total = df.shape[0]
train = df.iloc[:int(total * 0.75)]
test = df.iloc[int(total * 0.75):]

In [76]:
STOCK = 5
X_train, y_train = split_seq(train, STOCK)
X_train.shape

(641, 20, 12)

In [77]:
X_train

array([[[ 5.47438536e+01,  5.22925710e+01,  3.19560076e-02, ...,
         -2.78164117e-01, -1.25000000e+01, -2.53333333e+01],
        [ 5.83344507e+01,  5.38667444e+01,  3.69707443e-02, ...,
          4.18994413e-01, -0.00000000e+00, -2.72727273e+01],
        [ 5.42867625e+01,  5.17990665e+01,  3.13439704e-02, ...,
          0.00000000e+00, -0.00000000e+00, -4.85148515e+01],
        ...,
        [ 4.04492599e+01,  4.82606394e+01, -6.22870215e-02, ...,
         -3.83408853e-01, -1.00000000e+02, -8.62385321e+01],
        [ 4.34619908e+01,  5.01561553e+01, -4.91329371e-02, ...,
         -6.23484586e-01, -9.43181818e+01, -7.61467890e+01],
        [ 4.88167641e+01,  5.34176972e+01, -2.91840410e-02, ...,
         -4.48275862e-01, -7.38636364e+01, -5.28455285e+01]],

       [[ 4.70943770e+01,  4.87322566e+01,  1.56480261e-02, ...,
         -6.62482566e-01, -4.21875000e+01, -4.21875000e+01],
        [ 5.47438536e+01,  5.22925710e+01,  3.19560076e-02, ...,
         -2.78164117e-01, -1.25000000e

In [78]:
X_train[0]

array([[ 5.47438536e+01,  5.22925710e+01,  3.19560076e-02,
         1.29118961e-02,  2.01197119e-02,  9.58333333e+01,
         3.39582713e+01,  1.30695867e+00, -1.10344828e+00,
        -2.78164117e-01, -1.25000000e+01, -2.53333333e+01],
       [ 5.83344507e+01,  5.38667444e+01,  3.69707443e-02,
         2.35914118e-02,  1.64491957e-02,  1.00000000e+02,
         2.74617539e+01,  5.94613501e-01, -1.27016821e+00,
         4.18994413e-01, -0.00000000e+00, -2.72727273e+01],
       [ 5.42867625e+01,  5.17990665e+01,  3.13439704e-02,
         2.22352626e-02,  4.15219123e-03,  9.44444444e+01,
         2.32487281e+01,  5.97119775e-01, -2.65125765e+00,
         0.00000000e+00, -0.00000000e+00, -4.85148515e+01],
       [ 5.42867625e+01,  5.17990665e+01,  2.90971283e-02,
         2.73903967e-02, -5.69679069e-03,  9.44444444e+01,
         2.02643923e+01,  9.16138125e-01, -2.98102981e+00,
         4.20757363e-01, -0.00000000e+00, -6.00000000e+01],
       [ 5.03823943e+01,  4.97671601e+01,  2.0477344

In [79]:
y_train

<tf.Tensor: shape=(641, 3), dtype=float32, numpy=
array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)>

In [80]:
y_train.numpy().sum(axis=0)

array([304.,   6., 331.], dtype=float32)

In [81]:
model = keras.models.Sequential()
model.add(layers.Input(shape=(X_train.shape[1], X_train.shape[2])))
model.add(layers.SimpleRNN(units=5, activation="relu", return_sequences=True))
model.add(layers.SimpleRNN(units=5, activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.Dense(3))

In [82]:
model.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer="adam",
    metrics=["accuracy"]
)

In [83]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_6 (SimpleRNN)    (None, 20, 5)             90        
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 5)                 55        
                                                                 
 batch_normalization_3 (Bat  (None, 5)                 20        
 chNormalization)                                                
                                                                 
 dense_3 (Dense)             (None, 3)                 18        
                                                                 
Total params: 183 (732.00 Byte)
Trainable params: 173 (692.00 Byte)
Non-trainable params: 10 (40.00 Byte)
_________________________________________________________________


In [84]:
y_train.shape

TensorShape([641, 3])

In [85]:
X_test, y_test = split_seq(test, STOCK)
X_test.shape

(141, 20, 12)

In [86]:
model.fit(X_train, y_train, batch_size=30, epochs=100, validation_data=[X_test, y_test])

Epoch 1/100
22/22 [==============================] - 2s 18ms/step - loss: 1.3825 - accuracy: 0.2824 - val_loss: 13.0864 - val_accuracy: 0.0071
Epoch 2/100
22/22 [==============================] - 0s 6ms/step - loss: 1.3207 - accuracy: 0.3245 - val_loss: 7.0544 - val_accuracy: 0.0071
Epoch 3/100
22/22 [==============================] - 0s 7ms/step - loss: 1.2581 - accuracy: 0.3370 - val_loss: 4.8231 - val_accuracy: 0.0071
Epoch 4/100
22/22 [==============================] - 0s 8ms/step - loss: 1.2044 - accuracy: 0.3651 - val_loss: 3.5195 - val_accuracy: 0.0284
Epoch 5/100
22/22 [==============================] - 0s 8ms/step - loss: 1.1458 - accuracy: 0.3916 - val_loss: 2.6798 - val_accuracy: 0.1773
Epoch 6/100
22/22 [==============================] - 0s 7ms/step - loss: 1.1087 - accuracy: 0.4165 - val_loss: 2.3290 - val_accuracy: 0.2624
Epoch 7/100
22/22 [==============================] - 0s 6ms/step - loss: 1.0745 - accuracy: 0.4212 - val_loss: 1.8480 - val_accuracy: 0.3121
Epoch 8/100

In [87]:
y_test

<tf.Tensor: shape=(141, 3), dtype=float32, numpy=
array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
     

In [88]:
y_pred = model.predict(X_test)

5/5 [==============================] - 0s 4ms/step


In [89]:
y_pred

array([[ 0.75026226, -3.009335  ,  0.88172394],
       [ 0.5935399 , -2.904694  ,  0.8976675 ],
       [ 0.46936142, -2.8880348 ,  0.8683809 ],
       [ 0.71156365, -2.7715511 ,  1.0378397 ],
       [ 0.83951896, -2.693008  ,  1.1433342 ],
       [ 0.92763263, -2.5985098 ,  1.23684   ],
       [ 0.97284263, -2.7194872 ,  1.1692433 ],
       [ 1.149591  , -2.8218803 ,  1.1624143 ],
       [ 1.4220581 , -2.9210792 ,  1.1937597 ],
       [ 1.7473078 , -2.9768543 ,  1.2759256 ],
       [ 1.8694043 , -2.9455547 ,  1.3462951 ],
       [ 1.8376219 , -2.757666  ,  1.4654431 ],
       [ 1.6700172 , -2.5872474 ,  1.521216  ],
       [ 1.0491991 , -2.4444876 ,  1.392717  ],
       [ 0.9925151 , -2.2661304 ,  1.4942832 ],
       [ 0.7935521 , -2.2261915 ,  1.4491739 ],
       [ 0.76612425, -2.3633971 ,  1.3409548 ],
       [ 0.8029171 , -2.6399064 ,  1.163785  ],
       [ 0.790996  , -2.7159777 ,  1.1072257 ],
       [ 0.76399857, -2.6472883 ,  1.1446757 ],
       [ 0.6781429 , -2.440383  ,  1.258

In [90]:
y_pred_label = np.argmax(y_pred, axis=1)
y_pred_label

array([2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)

In [91]:
y_test_label = np.argmax(y_test, axis=1)
y_test_label

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2,
       2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [92]:
print(accuracy_score(y_true=y_test_label, y_pred=y_pred_label))

0.6099290780141844
